In [ ]:
using Pkg
Pkg.activate("../Tambo/")
Pkg.instantiate()
using Tambo

In [ ]:
using Plots
using JLD2
using StaticArrays
using StatsBase

In [ ]:
const sim = jldopen("/Users/jlazar/Downloads/GraphNet_00000_00395.jld2")
const config = SimulationConfig(; Dict(k=>v for (k, v) in sim["config"] if k != :geo_spline_path)...)
const geo = Tambo.Geometry(config)
const plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
const altmin = 1.8925255158436627units.km
const altmax = 4.092525515843662units.km

## First, we have to load up the data, and make our array

In [ ]:
jldf = jldopen("/Users/jlazar/Downloads/event_dicts_00000_00001.jld2")
# jldf = jldopen("../resources/example_event_dicts.jld2")
key = "2000.0_100.0"
event_dicts = jldf[key]

array_length = parse(Float64, split(key, '_')[1])
spacing = parse(Float64, split(key, '_')[2])

modules = Tambo.make_detector_array(
    whitepaper_coord,
    array_length * units.m,
    spacing * units.m,
    altmin,
    altmax,
    plane,
    geo
);

## Utility functions

### Finds the time window for an event

In [ ]:
function find_time_range(hit_map)
    tmin, tmax = Inf, -Inf
    for (idx, evts) in hit_map
        for evt in evts
            tmin = minimum([evt.time, tmin])
            tmax = maximum([evt.time, tmax])
        end
    end
    return tmin, tmax
end

### These functions allow us to plot background contour map

In [ ]:
function plot_topographic_map!(
    plt,
    geo;
    xmin=-2.5units.km,
    xmax=2.5units.km,
    ymin=-2.5units.km,
    ymax=2.5units.km,
    contour_color=palette(:lapaz, rev=true)
    
)
    plot!(
        plt,
        size=(500, 500),
        xlimits=(xmin, xmax)./units.km,
        ylimits=(ymin, ymax)./units.km
    )
    
    contour_xs = LinRange(xmin, xmax, 201)
    contour_ys = LinRange(ymin, ymax, 200)
    
    contour!(
        plt,
        contour_xs ./ units.km,
        contour_ys ./ units.km,
        @. (geo(contour_xs', contour_ys) + geo.tambo_offset[3]) / units.km;
        fill=true,
        color=contour_color,
        clims=(-1.5+geo.tambo_offset[3]/units.km, 2+geo.tambo_offset[3]/ units.km)
    )
end

function plot_topographic_map(
    geo;
    xmin=-2.5units.km,
    xmax=2.5units.km,
    ymin=-2.5units.km,
    ymax=2.5units.km,
    contour_color=palette(:lapaz, rev=true)  
)
    plt = plot()
    plot_topographic_map!(plt, geo, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax, contour_color=contour_color)
    return plt
end



### Now we write some helper function for overlaying the modules on the contour map



In [ ]:
function plot_modules!(
    plt,
    modules,
    hit_map=nothing;
    time_cmap=cgrad(:rainbow, rev=true),
    module_color=colorant"black",
    tcut=Inf
)

    colors = fill(module_color, length(modules))
    markersize = fill(3.0, length(modules))
    
    if ~(hit_map==nothing)
        tmin, tmax = find_time_range(hit_map)
        hit_color(t) = get(time_cmap, (t - tmin) / (tmax - tmin))
        ts = LinRange(tmin, tmax, 100)
        for (idx, evts) in hit_map
            evts = filter(e->e.time<tcut, evts)
            if length(evts)==0
                continue
            end
            colors[idx] = hit_color(mean(getfield.(evts, :time)))
            markersize[idx] = 3 + log(length(evts))
        end
    end

    scatter!(
        plt,
        getindex.(getfield.(modules, :pos), 1) ./ units.km,
        getindex.(getfield.(modules, :pos), 2) ./ units.km,
        alpha=0.75,
        markersize=markersize,
        color=colors,
        markerstrokewidth=0,
        label=""
    )
end

function plot_modules(
    modules,
    hit_map=nothing;
    time_cmap=cgrad(:rainbow, rev=true),
    module_color=colorant"black",
    tcut=Inf
)
    plt = plot()
    plot_modules!(
        plt,
        modules,
        hit_map=hit_map,
        time_cmap=time_cmap,
        module_color=module_color,
        tcut=tcut,
    )
    return plt
end

## Now we can piece these together to make the plot

Events of note are 5550, ...

In [ ]:

include("../scripts/trigger_defs.jl")

In [ ]:
event_dicts["21"]

In [ ]:
f = "../resources/example_hit_maps.jld2"

jldf = jldopen(f)
key = "2000.0_100.0"
event_dicts = jldf[key]
event_dicts

In [ ]:
f = "../resources/example_hit_maps.jld2"

jldf = jldopen(f)
key = "2000.0_100.0"
event_dicts = jldf[key]

for k in keys(event_dicts)
    event_number = parse(Int, k)
    hit_map = event_dicts[string(event_number)]
    if ~did_trigger(hit_map)
        continue
    end
    println(k)
    plt = plot_topographic_map(geo)
    plot_modules!(plt, modules, hit_map)
    display(plt)
end

## We can extend these to make gifs

In [ ]:
function plot_event_gif(
    geo,
    modules,
    hit_map;
    ntimes=100,
    xmin=-2.5units.km,
    xmax=2.5units.km,
    ymin=-2.5units.km,
    ymax=2.5units.km,
    color=palette(:lapaz, rev=true),
    time_cmap=cgrad(:rainbow, rev=true),
    module_color=colorant"black",
    filename="./tambo_event.gif",
    fps=15
)
    tmin, tmax = find_time_range(hit_map)
    tcuts = LinRange(tmin, tmax, ntimes)
    anim = @animate for tcut in tcuts
        plt = plot_topographic_map(geo)
        plot_modules!(plt, modules, hit_map, tcut=tcut)
    end
    gif(anim, filename, fps=fps)
end

In [ ]:
plot_event_gif(geo, modules, hit_map, fps=10)